In [1]:
import subprocess
import shlex
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]=""

In [ ]:
fn = "./examples/robust04/config/drmm_robust04.config"
for i in range(100, 5100, 100):
    f = open(fn)
    f2 = open(fn+"_tmp", "w")
    for l in f:
        if "test_weights_iters" in l:
            l = "\"test_weights_iters\": {}, \n".format(i)
        f2.write(l)
    f2.close()
    pargs = shlex.split("python matchzoo/main.py --phase predict --model_file ./examples/robust04/config/drmm_tweet.config_tmp")
    p = subprocess.Popen(pargs, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    pout, perr = p.communicate()
    pargs = shlex.split("./eval/trec_eval.9.0/trec_eval qrels.robust2004.txt data/robust04/predict.test.drmm.txt")
    p = subprocess.Popen(pargs, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    pout, perr = p.communicate()
    lines = pout.split(b'\n')
    if len(perr) > 0:
        print(perr)
    print(i)
    for l in lines:
        l = l.decode("utf8")
        for metric in ["map", "recip_rank", "P_30"]:
            if l.startswith(metric):
                print(l)